# get census raw data

In [54]:
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd
from io import BytesIO

In [55]:
storage_client = storage.Client()
bucket = storage_client.get_bucket('opengeokpi-bucket')
blob = bucket.blob('raw/population/base-cc-evol-struct-pop-2017.csv')
content =blob.download_as_string()
communes_census_2017 = pd.read_csv(BytesIO(content), sep = ';',dtype=str)

In [56]:
communes_census_2017['PopulationTotal'] = df['P17_POP'].astype(float)

In [57]:
# rename columns
communes_census_2017 = communes_census_2017.rename(columns={
                                  "CODGEO": "Code"})

In [58]:
communes_census_2017 = communes_census_2017[['Code','PopulationTotal']]

In [59]:
communes_census_2017.head()

,Code,PopulationTotal
0,01001,776.0
1,01002,248.0
2,01004,14035.0
3,01005,1689.0
4,01006,111.0


# save refine data

In [52]:
# save in work environment
with open('data/population/communes_census_2017.csv', 'w') as f:
    f.write(communes_census_2017.to_csv())

In [53]:
# upload in google storage
blob = bucket.blob('refine/population/communes_census_2017.csv')

communes_census_2017  = 'data/population/communes_census_2017.csv'
blob.upload_from_filename(communes_census_2017)

print("File uploaded to {}.".format(bucket.name))

File uploaded to opengeokpi-bucket.


In [4]:
# get communes geometry data

In [7]:
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

query = """
    SELECT *
    FROM `test-rstudio-gcp.opengeokpi.zones_communes_geo`
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

zones_communes_geo = query_job.to_dataframe()
zones_communes_geo.head()

Client creating using default project: test-rstudio-gcp


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:407: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,geom_communes,communes_level,communes_code,communes_name,communes_population,communes_area
0,"POLYGON((-0.269573 43.205079, -0.26402 43.1980...",fr:commune,64101,Baudreix,581.0,2.0
1,"POLYGON((-4.071203 48.220714, -4.076096 48.224...",fr:commune,29222,Port-Launay,389.0,2.0
2,"POLYGON((0.151986 43.247954, 0.163869 43.24715...",fr:commune,65225,Hourc,111.0,2.0
3,"POLYGON((2.084783 43.137604, 2.086874 43.13348...",fr:commune,11167,Gramazie,113.0,2.0
4,"POLYGON((4.369932 45.646579, 4.371724 45.64775...",fr:commune,42335,Viricelles,466.0,2.0


In [17]:
#zones_communes_geo['communes_code'].values.tolist()

In [13]:
census_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34996 entries, 0 to 34995
Columns: 312 entries, CODGEO to C07_POP55P_CS8
dtypes: float64(311), object(1)
memory usage: 83.3+ MB


In [25]:
commune_kpi_population = pd.merge(zones_communes_geo, census_2017, left_on='communes_code', right_on='CODGEO')

In [26]:
commune_kpi_population.head()

,geom_communes,communes_level,communes_code,communes_name,communes_population,communes_area,CODGEO,P17_POP,P17_POP0014,P17_POP1529,...,C07_POP55P,C07_POP55P_CS1,C07_POP55P_CS2,C07_POP55P_CS3,C07_POP55P_CS4,C07_POP55P_CS5,C07_POP55P_CS6,C07_POP55P_CS7,C07_POP55P_CS8,Population
0,"POLYGON((-0.269573 43.205079, -0.26402 43.1980...",fr:commune,64101,Baudreix,581.0,2.0,64101,611,106.886244825678,155.225052719982,...,81.547031952713,0,0,4.07735159763565,4.07735159763565,0,0,61.1602739645348,12.232054792907,611.0
1,"POLYGON((-4.071203 48.220714, -4.076096 48.224...",fr:commune,29222,Port-Launay,389.0,2.0,29222,391,46.1972573702485,74.6471340451079,...,149.550128915545,0,4.27286082615844,4.27286082615844,17.0914433046338,4.27286082615844,17.0914433046338,76.9114948708519,25.6371649569506,391.0
2,"POLYGON((0.151986 43.247954, 0.163869 43.24715...",fr:commune,65225,Hourc,111.0,2.0,65225,109,20.7619047619048,9.34285714285716,...,11.8211395299141,0,0,3.9403798433047,0,0,0,7.8807596866094,0,109.0
3,"POLYGON((2.084783 43.137604, 2.086874 43.13348...",fr:commune,11167,Gramazie,113.0,2.0,11167,117,27.2327586206896,14.1206896551724,...,29.2824631394306,0,0,0,0,0,0,20.9160450995933,8.36641803983732,117.0
4,"POLYGON((4.369932 45.646579, 4.371724 45.64775...",fr:commune,42335,Viricelles,466.0,2.0,42335,457,101.325791855204,58.9343891402715,...,107.577336631709,0,3.7095633321279,0,0,7.4191266642558,3.7095633321279,77.9008299746859,14.8382533285116,457.0


In [24]:
df_merge_difkey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35494 entries, 0 to 35493
Columns: 319 entries, geom_communes to Population
dtypes: float64(3), object(316)
memory usage: 86.7+ MB
